## Building a symbolic classifier

MDL based algorithms encode data according to a given codetable

When calling ``.fit``, we iteratively look for the codetable that compress
the training data the best

**When we are done with training our model, we can benefit from the refined codetable 
to make some predictions**

In [1]:
import pandas as pd
from skmine.itemsets import SLIM

In [2]:
In [1]: D = pd.Series([  # SLIM takes a pd.Series as input 
     ['bananas', 'milk'], 
     ['milk', 'bananas', 'cookies'], 
     ['cookies', 'butter', 'tea'], 
     ['tea'],  
     ['milk', 'bananas', 'tea'], 
]) 
D 

0             [bananas, milk]
1    [milk, bananas, cookies]
2      [cookies, butter, tea]
3                       [tea]
4        [milk, bananas, tea]
dtype: object

In [3]:
slim = SLIM()
slim.fit(D)

(milk, bananas, cookies)       [1]
(cookies, tea, butter)         [2]
(milk, bananas)             [0, 4]
(tea)                       [3, 4]
dtype: object

----------
We define a new transactional dataset, we some unseen items inside
and call the ``predict_proba`` function. This basically computes the probability
of a given transaction to belong to the current codetable (Shannon Entropy)

In [4]:
new_D = pd.Series([  # SLIM takes a pd.Series as input 
   ['bananas', 'milk'], 
   ['milk', 'sirup', 'cookies'], 
   ['butter', 'tea'], 
   ['tea'],  
   ['milk', 'bananas', 'tea'], 
]) 
new_D 

0           [bananas, milk]
1    [milk, sirup, cookies]
2             [butter, tea]
3                     [tea]
4      [milk, bananas, tea]
dtype: object

In [5]:
slim.predict_proba(new_D)

0    0.333333
1    0.000000
2    0.333333
3    0.333333
4    0.666667
dtype: float64